## Código de preparación de datos...

Este pequeño código prepara los datos para ser manipulados y ser usados por otro código de propagación de muones en roca.. La idea es que tenemos dos archivos de datos. El primero es la salida de muones del volcán machin en el punto P1 de observación, el otro archivo de datos es la distancia recorrida por los muones en roca, calculado a partir de un código en matlab externo. La idea es manipular el primero de los archivos para obtener la energía total a partir de la consideración de los momentos en las direcciones $x$, $y$, y $z$. Lo segundo es teniendo la energía vamos a organizar los datos de tal manera de calcular el flujo, es decir, sumar todos los muones que llegan en la misma dirección para tener el número de muones que entran por esa determinada dirección, con esto tendremos una data de flujo por dirección.
Lo siguiente es combinar el archivo de datos de distancias recorridas y le ponga a cada partícula entrante con energía determinada E, su correspondiente distancia recorrida...

In [1]:
# Cargando todas las librerías que necesitamos
import numpy as np
import math
import pandas as pd 
import matplotlib.pyplot as plt 

#ecuaciones de pérdida de energía 


def funciondEdX(E): 
    me = 0.0005285 #GeV/c2
    mu = 0.1057 #GeV/c2
    
    
    Emax = (E**2)/(E+(mu**2)/(2*me))
    dEdx = 1.84 + 0.076* np.log(Emax/(mu))

    return dEdx
        
        
def perdida_energia_total(E, d): #energia y distancia por recorrer
    
    
    temp = E
    ar_x = np.linspace(0, d, 100)
    
    for i in range(len(ar_x)): 
        
        temp = temp - funciondEdX(temp)
        
        if temp < 0 : 
            control = False
            temp = 0
            break 
        control = True
        
    energia_final = temp 
    
    
    return (energia_final, control)
    

## Diccionario de distancias 

def acceder_diccionario(diccionario, valor_clave):
    valor_distancia = 0 
    for clave, valor in diccionario.items(): 
        if clave == valor_clave: 
            valor_distancia = diccionario[valor_clave]
    return valor_distancia

#####################
    
        
theta1, theta2= 18, 34 #<--------------- estos valores son importantes y sales de la apertura
phi1 , phi2 = 89, 91
############################
df1=pd.read_csv('./salida_d_2.shw',sep =" ", header=None)
df1=df1.drop(index=range(6))  
df2 = df1[df1.columns[1:4]] 
px = df2[df2.columns[:1]].to_numpy()
py = df2[df2.columns[1:2]].to_numpy()
pz = df2[df2.columns[2:3]].to_numpy()
pt=np.sqrt(px**2+py**2+pz**2) 

Llamamos al primer archivo de datos originales que vienen de Corsika...

Acá ahora tomamos los momentos y calculamos la energía total de cada muon que llega al volcán... En este caso 
\begin{equation}
\theta=\arccos\frac{p_z}{p}=\arccos\frac{p_z}{\sqrt{p_x^2+p_y^2+p_z^2}}, \ \ \phi=\arctan\frac{p_y}{p_x}
\end{equation}

In [2]:
pt=np.sqrt(px**2+py**2+pz**2)
m_rest=0.1057 #unit: GeV/c**2
Et=np.sqrt(pt**2+m_rest**2)
thetat=np.degrees(np.arccos(pz/pt))
at2=np.arctan2(py,px)
thetat=np.round(np.degrees(np.arccos(pz/pt)))
at2=np.arctan2(py,px)
phit=np.round(np.degrees((2*np.pi+at2)*(at2<0)+(at2)*(at2>0)))

#Ahora tenemos un archivo de ángulos de incidencia vs. energía con la que llegan. 
#Guardamos los datos nuevos con el formato requerido... Cargamos la nueva dataframe y ordenamos por ángulos.

data=np.c_[phit,thetat,Et] #theta ya esta en grados
data=data[data[:,1].argsort()] # Ordena de acuerdo a angulo theta  

# Obtenemos solo los que están en el rango phi1<phi<phi2 y theta1<theta<theta2
data=data[(data[:,0]== 90) & (data[:,1]>=theta1) & (data[:,1]<=theta2)] #--- acá le pongo la condición 
#data=np.savetxt('totalrealCerroUNI_tpE_ordenados.out',data,delimiter=' ',fmt='%i %i %2f')
dfshower_filtrado = pd.DataFrame(data, columns=['phi','theta','E'])
#conteo de los valores a cielo abierto
dfshower_conteo=dfshower_filtrado.groupby(['phi','theta'])['E'].count().reset_index(name='count')
dfshower_conteo.to_csv('totalrealCerroUNI_tpcount_ordenados_cielo_abierto.out',sep=' ',index=False) 
df_distancia=pd.read_csv('datos_fila_210_angmax_48_apertura_30_pasos_19.txt',sep=' ',names=['fila','angulo','distancia'],skiprows=1)
df_distancia['Elevacion'] = 90-df_distancia['angulo'] 

#####
vec_distancia = df_distancia[df_distancia.columns[2:4]].to_numpy() #distancias a observar
diccionario = {}  # Inicializar el diccionario vacío

for i in range(0, len(df_distancia)):  # Bucle for para generar claves del 1 al 5
    clave = vec_distancia[i][1] #valores de distancia
    valor = vec_distancia[i][0] #valores de ángulos 
    diccionario[clave] = valor  # Agregar clave-valor al diccionario 

# Calculamos la atenuación para cada uno de los eventos 
# obtenemos al final el vector veco con la siguiente información para todos los eventos
#res_atenua = phi , theta, distancia recorrida, Einicial, Efinal, Control

fac = 1000 # Conversión MeV to GeV
veco=[]
vec_energia = dfshower_filtrado[dfshower_filtrado.columns[2:3]].to_numpy() #

for j in range(len(dfshower_filtrado)):     
    temp_angulo_shower = dfshower_filtrado.iloc[j][1] # ángulo theta del evento j en el shower 
    temp_energia_shower = dfshower_filtrado.iloc[j][2] * fac # energía inicial del evento 
    temp_distancia_x_angulo = acceder_diccionario(diccionario, temp_angulo_shower)
    #print(temp_distancia_x_angulo, temp_energia_shower, temp_angulos_shower) funciona
    #print(funciondEdX(temp_energia_shower)) funciona
    val_temp = perdida_energia_total(temp_energia_shower, temp_distancia_x_angulo)[1] 
    #res_atenua = phi , theta, distancia recorrida, Einicial, Efinal, Control
    res_atenua = [dfshower_filtrado.iloc[j][0], temp_angulo_shower, temp_distancia_x_angulo, temp_energia_shower, perdida_energia_total(temp_energia_shower, temp_distancia_x_angulo)[0], val_temp] 
    veco.append(res_atenua)

# filtramos por valores de control si pasó o no el evento
# df_atenuado_filtro tiene el valor de theta para aquellos que pasaron y los que no
df_atenuado = pd.DataFrame(veco, columns=['phi','theta','distancia', 'Energia_i', 'Energía_f', 'Control'])
df_atenuado_con_filtro=df_atenuado.groupby(['theta','Control'])['theta'].count().reset_index(name='count') 

## Calculamos el porcentaje de atenuación

vec_atenuado = []
for j in range(0,len(df_atenuado_con_filtro)-1,2):     
    angle_temp = df_atenuado_con_filtro.iloc[j][0] 
    a = df_atenuado_con_filtro.iloc[j][2] 
    b = df_atenuado_con_filtro.iloc[j+1][2] 
    por = np.round(b/(a+b)*100,2)
    resultados = [angle_temp, por]
    vec_atenuado.append(resultados)

vec_atenuado    = np.array(vec_atenuado) 
print(vec_atenuado)

[[18.   22.03]
 [19.   20.97]
 [20.   23.19]
 [21.   44.12]
 [22.   29.58]
 [23.   26.98]
 [24.   35.48]
 [25.   23.17]
 [26.   23.29]
 [27.   32.91]
 [28.   34.88]
 [29.   32.94]
 [30.   34.21]
 [31.   34.38]
 [32.   31.08]
 [33.   28.4 ]
 [34.   29.85]]
